In [1]:

import pandas as pd
from pathlib import Path

excel_path = Path(r"/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/test_assets/collection/business-accounting/Business Accounting.xlsx")
df = pd.read_excel(excel_path, sheet_name=9)
print(f"Loaded sheet with shape: {df.shape}")

# Quick preview for LLM context
print("\nFirst 10 rows:")
print(df.head(10).to_string())
print("\nLast 5 rows:")
print(df.tail(5).to_string())

# Get basic info
sheet_name = "None"
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"


Loaded sheet with shape: (89, 8)

First 10 rows:
        Date                               Description    Amount  Column 1  Category   Tag   Total  -105863.94
0 2024-04-07  Appraisal - BNKP Property Solutions, LLC   -199.00       EXP       Fee   NaN     NaN         NaN
1 2024-04-11                              Down deposit -66999.89       EXP       NaN   NaN     NaN         NaN
2 2024-04-11                         wire transfer fee    -30.00       EXP       Fee   NaN     NaN         NaN
3 2024-04-30                                M3 Roofing -23500.00       EXP     Rehab   NaN     NaN         NaN
4 2024-04-30                                   ACH Fee     -5.00       EXP       Fee   NaN     NaN         NaN
5 2024-05-29                              Wildcat Draw  15487.00      LOAN       NaN   NaN     NaN         NaN
6 2024-05-29           Inspection Fee for draw request   -125.00       EXP       Fee   NaN     NaN         NaN
7 2024-05-29                                M3 Roofing  -8550.0

In [2]:

excel_file_name = excel_path.name
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"
sheet_name


'None'

In [3]:

# Table Detection Analysis
print("=== DATA STRUCTURE ANALYSIS ===")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 10 ROWS ===")
print(df.head(10).to_string())
print("\n=== LAST 5 ROWS ===")
print(df.tail(5).to_string())

print("\n=== COLUMN ANALYSIS ===")
# Check for empty columns that might separate tables
for i, col in enumerate(df.columns):
    non_null_count = df[col].notna().sum()
    print(f"Column {i} ('{col}'): {non_null_count}/{len(df)} non-null values")

print("\n=== EMPTY ROW ANALYSIS ===")
# Check for empty rows that might separate tables
empty_rows = df.isnull().all(axis=1)
if empty_rows.any():
    empty_row_indices = empty_rows[empty_rows].index.tolist()
    print(f"Empty rows found at indices: {empty_row_indices[:10]}")
else:
    print("No completely empty rows found")

print("\n=== SIDE-BY-SIDE ANALYSIS ===")
# Check for potential side-by-side tables by examining column groups
left_cols = df.iloc[:, 0:7]  # First 7 columns
right_cols = df.iloc[:, 7:]  # Remaining columns
print(f"Left section shape: {left_cols.shape}")
print(f"Right section shape: {right_cols.shape}")
print(f"Left section non-null density: {left_cols.notna().sum().sum() / (left_cols.shape[0] * left_cols.shape[1]):.2f}")
print(f"Right section non-null density: {right_cols.notna().sum().sum() / (right_cols.shape[0] * right_cols.shape[1]):.2f}")


=== DATA STRUCTURE ANALYSIS ===
DataFrame shape: (89, 8)
Columns: ['Date', 'Description', 'Amount', 'Column 1', 'Category', 'Tag', 'Total', -105863.93999999999]

=== FIRST 10 ROWS ===
        Date                               Description    Amount  Column 1  Category   Tag   Total  -105863.94
0 2024-04-07  Appraisal - BNKP Property Solutions, LLC   -199.00       EXP       Fee   NaN     NaN         NaN
1 2024-04-11                              Down deposit -66999.89       EXP       NaN   NaN     NaN         NaN
2 2024-04-11                         wire transfer fee    -30.00       EXP       Fee   NaN     NaN         NaN
3 2024-04-30                                M3 Roofing -23500.00       EXP     Rehab   NaN     NaN         NaN
4 2024-04-30                                   ACH Fee     -5.00       EXP       Fee   NaN     NaN         NaN
5 2024-05-29                              Wildcat Draw  15487.00      LOAN       NaN   NaN     NaN         NaN
6 2024-05-29           Inspection Fee f

In [4]:
detected_tables = []

# Based on the analysis, the data appears to be a single table with no clear mechanical or side-by-side separation.
# The shape is (89, 8), and no empty rows are found, indicating a continuous dataset.
# The columns suggest a single structured table with some columns possibly less populated.

# Define the main table boundaries:
start_row = 0
end_row = 88  # zero-indexed, inclusive
start_col = 0
end_col = 7  # zero-indexed, inclusive

detected_tables.append({
    "table_id": "table_1",
    "description": "Financial transactions data",
    "start_row": start_row,
    "end_row": end_row,
    "start_col": start_col,
    "end_col": end_col,
    "confidence": 0.95,
    "table_type": "DETAIL",
    "entity_type": "transactions"
})

# No additional tables detected based on the analysis, so return the list as is.
detected_tables

[{'table_id': 'table_1',
  'description': 'Financial transactions data',
  'start_row': 0,
  'end_row': 88,
  'start_col': 0,
  'end_col': 7,
  'confidence': 0.95,
  'table_type': 'DETAIL',
  'entity_type': 'transactions'}]

In [5]:

# Extract detection results created by LLM
# The detector should have created a 'detected_tables' variable with the results

# CLAUDE-TEST-WORKAROUND: Validate that detected_tables exists and is properly formatted
if 'detected_tables' in globals():
    # Validate it's a list
    if isinstance(detected_tables, list):
        detection_results = detected_tables
        print(f"✅ Found {len(detection_results)} tables from LLM detection")
        # Validate first table has required fields (if any tables exist)
        if detection_results:
            required_fields = ['table_id', 'description', 'start_row', 'end_row', 'start_col', 'end_col']
            first_table = detection_results[0]
            missing_fields = [f for f in required_fields if f not in first_table]
            if missing_fields:
                print(f"⚠️ Warning: First table missing fields: {missing_fields}")
    else:
        print(f"❌ Error: detected_tables is not a list, it's a {type(detected_tables)}")
        detection_results = []
else:
    # CLAUDE-GOTCHA: Gemini sometimes fails to create the variable even after multiple prompts
    print("❌ No 'detected_tables' variable found - LLM failed to complete detection")
    detection_results = []

detection_results


✅ Found 1 tables from LLM detection


[{'table_id': 'table_1',
  'description': 'Financial transactions data',
  'start_row': 0,
  'end_row': 88,
  'start_col': 0,
  'end_col': 7,
  'confidence': 0.95,
  'table_type': 'DETAIL',
  'entity_type': 'transactions'}]


# Table Detection Results

Detected 1 tables:


## Table 1: Financial transactions data
- Location: Rows 0-88, Columns 0-7
- Type: detail
- Entity: transactions
- Confidence: 0.95
